# **REACTION: Breast Cancer Detection**









Read your mamography image file and provide an alternative result for breast cancer diagnose using image preprocesses 

(grayscale, normalize, and gaussian filter)

In [73]:
from ipywidgets import Layout, Button, Box, FileUpload, Label, Checkbox, IntSlider, IntText, Button
import ipywidgets as widgets
from IPython.display import display
import threading
import time

# uploader to image cv2
import numpy as np
import cv2
import matplotlib.pyplot as plt
 
import os

In [74]:
#define flag
thread_status = True

In [75]:
# uploader
uploader = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    )

In [76]:
# uploader container
image_container_raw = widgets.VBox(
    children = ()
)

Exception in thread work:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 2252, in validate_elements
    v = self._trait._validate(obj, v)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 591, in _validate
    value = self.validate(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1677, in validate
    self.error(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1524, in error
    raise TraitError(e)
traitlets.traitlets.TraitError: The 'None' trait of a VBox instance must be a Widget, but a value of class 'NoneType' (i.e. None) was specified.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, 

In [77]:
# wrapper
app = widgets.VBox(
    children = (uploader, image_container_raw)
)

In [78]:
# uploader image content and extension
def get_content_and_extension(uploader):    
    for _, value in uploader.value.items():
        return value['content'], value['metadata']['name'].split('.')[-1]

In [79]:
# image widget from uploader
def image_widget_from_uploader(uploader):
    if uploader.value:
        file_content, file_extension = get_content_and_extension(uploader)
        return widgets.Image(value=file_content, format=file_extension)

In [80]:
# image numpy from uploader
def image_numpy_from_uploader(uploader):
    file_content, extension = get_content_and_extension(uploader)
    arr_from_buf = np.frombuffer(file_content, np.uint8)
    image_numpy = cv2.imdecode(arr_from_buf, cv2.IMREAD_COLOR)
    return image_numpy, extension

In [81]:
# image widget from image numpy
def image_widget_from_numpy(image_numpy, extension):
    _, encoded_image = cv2.imencode('.'+extension, image_numpy,)
    image_bytes = encoded_image.tobytes()
    return widgets.Image(value=image_bytes, format=extension)

In [82]:
# individual preprocessing
def prepro_greyscale(image_numpy):
    image_numpy = cv2.cvtColor(image_numpy, cv2.COLOR_BGR2GRAY)
    return image_numpy

In [83]:
def prepro_normalize(image_numpy,resize_w=512,resize_l=512):
    out_arr = np.zeros((resize_w,resize_l))
    image_numpy = cv2.normalize(image_numpy, out_arr, 0, 255, cv2.NORM_MINMAX)
    return image_numpy

In [84]:
def prepro_gaussian(image_numpy, kernel_size):
    img_numpy = cv2.GaussianBlur(image_numpy, (kernel_size, kernel_size), 0)
    noise = img_numpy - image_numpy
    filtered = image_numpy - noise
    return  filtered

In [85]:
def preprocessing(kernel_size, image_numpy,to_greyscale=True,resize_w=512,resize_l=512,to_normalize=True, to_gaussian_filter=True):
    if to_greyscale:
        image_numpy= prepro_greyscale(image_numpy)
    if to_normalize:
        image_numpy= prepro_normalize(image_numpy,resize_w=resize_w,resize_l=resize_l)
    if to_gaussian_filter:
        image_numpy = prepro_gaussian(image_numpy, kernel_size)
    return image_numpy

In [86]:
slider_ksize = widgets.IntSlider(
    value=5,
    min=1,
    max=100,
    step=2,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    )

In [87]:
checkbox_greyscale = widgets.Checkbox(
    value=True,
    description='greyscale',
    disabled=False,
    indent=False
)
checkbox_normalize = widgets.Checkbox(
    value=True,
    description='normalize',
    disabled=False,
    indent=False
)
checkbox_gaussian_filter = widgets.Checkbox(
    value=True,
    description='gaussian',
    disabled=False,
    indent=False
)
checkbox_greyscale_normalize = widgets.Checkbox(
    value=True,
    description='greyscale + normalize',
    disabled=False,
    indent=False
)
checkbox_greyscale_gaussian = widgets.Checkbox(
    value=True,
    description='greyscale + gaussian',
    disabled=False,
    indent=False
)
checkbox_normalize_gaussian = widgets.Checkbox(
    value=True,
    description='normalize + gaussian',
    disabled=False,
    indent=False
)
checkbox_greyscale_normalize_gaussian = widgets.Checkbox(
    value=True,
    description='greyscale + normalize + gaussian',
    disabled=False,
    indent=False
)

# display(checkbox_greyscale)
# display(checkbox_normalize)
# display(checkbox_gaussian_filter)

In [88]:
checkboxes = widgets.HBox(
    children = (checkbox_greyscale, checkbox_normalize, checkbox_gaussian_filter, checkbox_greyscale_normalize, checkbox_greyscale_gaussian, checkbox_normalize_gaussian, checkbox_greyscale_normalize_gaussian)
)
# display(checkboxes)

In [89]:
image_container_prepro = widgets.VBox(
    children = ()
)

In [90]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Upload Mammography'), app], layout=form_item_layout),
    Box(children = (checkboxes,), layout=form_item_layout),
    Box([Label(value='Gaussian Filter Kernel Size'), slider_ksize], layout=form_item_layout),
    Box([Label(value='Visualization'), image_container_prepro], layout=form_item_layout)
]

form = widgets.VBox(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
  width='100%'
))
# display(form)

In [91]:
def image_widget_to_labeled(image_numpy, message):
    layout_img=widgets.Layout(
        display='flex',
        flex_flow='column',
        border='solid 2px',
        align_items='stretch',
        width='50%'
        )
    return widgets.VBox([widgets.Label(value=message), image_numpy, layout_img])   

In [92]:
# label
label_current_thread = widgets.Label()
label_thread_list = widgets.Label()

# button 
button_stop = widgets.Button(
    description = 'stop',
    icon = 'fa-stop',
    button_style = 'warning',
    layout = widgets.Layout(width='100px')
)
def button_stop_click(self):
    global thread_status
    thread_status = False
button_stop.on_click(button_stop_click)

button_check_thread = widgets.Button(
    description = 'check thread',
    button_style = '',
    layout = widgets.Layout(width='100px')
)
def button_check_thread_click(self):
    # print if work() still running
    work_count = 0
    name='work'
    for thread_object in threading.enumerate():
        if thread_object.name == name:
            work_count+=1
    if work_count > 0:
        label_thread_list.value=f'{work_count} thread with name {name} still exist. Consider use exit() and restart kernel'
    else:
        label_thread_list.value=f'No thread with name {name} exist'
button_check_thread.on_click(button_check_thread_click)

In [93]:
# main loop
def work():

    global thread_status
    
    if thread_status:
        label_current_thread.value = 'Thread On'

    current_state =[
        checkbox_greyscale.value,
        checkbox_normalize.value,
        checkbox_gaussian_filter.value,
        checkbox_greyscale_normalize.value,
        checkbox_greyscale_gaussian.value, 
        checkbox_normalize_gaussian.value,
        checkbox_greyscale_normalize_gaussian.value,
        slider_ksize.value,

        uploader.value
        ]
    
    while thread_status:
        
        # check if there is a change in uploader.value
        if current_state != [
            checkbox_greyscale.value,
            checkbox_normalize.value,
            checkbox_gaussian_filter.value,
            checkbox_greyscale_normalize.value,
            checkbox_greyscale_gaussian.value, 
            checkbox_normalize_gaussian.value,
            checkbox_greyscale_normalize_gaussian.value,
            slider_ksize.value,

            uploader.value
            ]:
            
            # renew state reference
            current_state = [
                checkbox_greyscale.value,
                checkbox_normalize.value,
                checkbox_gaussian_filter.value,
                checkbox_greyscale_normalize.value,
                checkbox_greyscale_gaussian.value, 
                checkbox_normalize_gaussian.value,
                checkbox_greyscale_normalize_gaussian.value,
                slider_ksize.value,
                
                uploader.value
                ]
            
            
            # image widget from uploader
            image_widget_raw = image_widget_from_uploader(uploader)
            
            # renew container_raw
            image_container_raw.children = (image_widget_raw,)

            # read image numpy
            image_numpy, extension = image_numpy_from_uploader(uploader)


            # preprocess numpy
            if checkbox_greyscale.value:
                image_numpy_grayscale = prepro_greyscale(image_numpy)
            if checkbox_normalize.value:
                image_numpy_normalize = prepro_normalize(image_numpy,resize_w=512,resize_l=512)
            if checkbox_gaussian_filter.value:
                image_numpy_gaussian  = prepro_gaussian(image_numpy, slider_ksize.value)
            if checkbox_greyscale_normalize.value:
                image_numpy_greyscale_normalize = preprocessing(slider_ksize.value, image_numpy,to_greyscale=True,resize_w=512,resize_l=512,to_normalize=True,to_gaussian_filter=False)
            if checkbox_greyscale_gaussian.value:
                image_numpy_greyscale_gaussian = preprocessing(slider_ksize.value, image_numpy,to_greyscale=True,resize_w=512,resize_l=512,to_normalize=False,to_gaussian_filter=True)
            if checkbox_normalize_gaussian.value:
                image_numpy_normalize_gaussian = preprocessing(slider_ksize.value, image_numpy,to_greyscale=False,resize_w=512,resize_l=512,to_normalize=True,to_gaussian_filter=True)
            if checkbox_greyscale_normalize_gaussian.value:
                image_numpy_greyscale_normalize_gaussian = preprocessing(slider_ksize.value, image_numpy,to_greyscale=True,resize_w=512,resize_l=512,to_normalize=True,to_gaussian_filter=True)


            #numpy to widget
            if checkbox_greyscale.value:
                image_widget_grayscale = image_widget_from_numpy(image_numpy_grayscale, extension)
            if checkbox_normalize.value:
                image_widget_normalize = image_widget_from_numpy(image_numpy_normalize, extension)
            if checkbox_gaussian_filter.value:
                image_widget_gaussian  = image_widget_from_numpy(image_numpy_gaussian, extension)
            if checkbox_greyscale_normalize.value:
                image_widget_greyscale_normalize  = image_widget_from_numpy(image_numpy_greyscale_normalize, extension)
            if checkbox_greyscale_gaussian.value:
                image_widget_greyscale_gaussian  = image_widget_from_numpy(image_numpy_greyscale_gaussian, extension)
            if checkbox_normalize_gaussian.value:
                image_widget_normalize_gaussian  = image_widget_from_numpy(image_numpy_normalize_gaussian, extension)
            if checkbox_greyscale_normalize_gaussian.value:
                image_widget_greyscale_normalize_gaussian = image_widget_from_numpy(image_numpy_greyscale_normalize_gaussian, extension)
            

                
            # image widget to labeled widget
            if checkbox_greyscale.value:
                labeled_image_greyscale = image_widget_to_labeled(image_widget_grayscale, 'greyscale')
            if checkbox_normalize.value:
                labeled_image_normalize = image_widget_to_labeled(image_widget_normalize, 'normalize')
            if checkbox_gaussian_filter.value:
                labeled_image_gaussian = image_widget_to_labeled(image_widget_gaussian, 'gaussian filter')
            if checkbox_greyscale_normalize.value:
                labeled_image_greyscale_normalize = image_widget_to_labeled(image_widget_greyscale_normalize, 'greyscale and normalize')
            if checkbox_greyscale_gaussian.value:
                labeled_image_greyscale_gaussian = image_widget_to_labeled(image_widget_greyscale_gaussian, 'greyscale and gaussian')
            if checkbox_normalize_gaussian.value:
                labeled_image_normalize_gaussian = image_widget_to_labeled(image_widget_normalize_gaussian, 'normalize and gaussian')
            if checkbox_greyscale_normalize_gaussian.value:
                labeled_image_greyscale_normalize_gaussian = image_widget_to_labeled(image_widget_greyscale_normalize_gaussian, 'greyscale, normalize, and gaussian')

            
            temp_list = []
            if checkbox_greyscale.value:
                temp_list.append(labeled_image_greyscale)
            if checkbox_normalize.value:
                temp_list.append(labeled_image_normalize)
            if checkbox_gaussian_filter.value:
                temp_list.append(labeled_image_gaussian)
            if checkbox_greyscale_normalize.value:
                temp_list.append(labeled_image_greyscale_normalize)
            if checkbox_greyscale_gaussian.value:
                temp_list.append(labeled_image_greyscale_gaussian)
            if checkbox_normalize_gaussian.value:
                temp_list.append(labeled_image_normalize_gaussian)
            if checkbox_greyscale_normalize_gaussian.value:
                temp_list.append(labeled_image_greyscale_normalize_gaussian)

            image_container_prepro.children = (*temp_list,)

       
        # sleep for threading
        time.sleep(0.05)
        
        if not thread_status:
            label_current_thread.value = 'Thread Off'

In [94]:
# restart flag
thread_status = False  
thread_status = True

# create thread
thread_work = threading.Thread(target=work)
thread_work.name = 'work'

# start thread
thread_work.start()

In [95]:
# display(button_check_thread)
# display(button_stop)
# display(label_thread_list)

In [96]:
form